In [1]:
# modified from https://www.kaggle.com/code/rohanlone4/plant-leaf-and-disease-recognition-using-fast-ai
# https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
 pip install fastai --upgrade 


In [2]:
from fastai.vision.all import *

Data Import

In [3]:
path='../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)'

In [4]:
fns=get_image_files(path)
fns

# Data generator

In [5]:
leaf=DataBlock(blocks=(ImageBlock,CategoryBlock),
               get_items=get_image_files,
               splitter=RandomSplitter(valid_pct=(0.2),seed=42),
               get_y=parent_label,item_tfms=Resize(224))

In [6]:
dls=leaf.dataloaders(path,num_workers = 0,bs=64)

# Downloading Resnet18 pre-trained model

In [7]:
learner=cnn_learner(dls,resnet18,metrics=[accuracy, error_rate])

### Finding best learning rate

In [8]:
lr_min,lr_steep = learner.lr_find(suggest_funcs=(minimum,steep))

In [9]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

## Best learning rate for this data

In [10]:
lr_steep

## Modeling

In [11]:
learner.fine_tune(3,base_lr=lr_steep,cbs=[ShowGraphCallback()])

In [15]:
learner.export("/model.pkl")

In [17]:
interp = ClassificationInterpretation.from_learner(learner)

In [40]:
interp.plot_top_losses(10)

In [23]:
from PIL import Image

In [30]:
test_path = '../input/new-plant-diseases-dataset/test/test/'
test_files = get_image_files(test_path)

In [39]:
open_image(test_files[0])